In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt


In [3]:
# show head without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)

In [4]:
merged = pd.read_csv('../data/oh_merged_till_2020.csv')
# merged.head()

/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_32913/84288552.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  merged = pd.read_csv('../data/oh_merged_till_2020.csv')


In [4]:
merged["Voters_BirthDate"].isnull().sum()

np.int64(1996)

In [5]:
df = merged.copy()

df.dropna(subset=["Voters_BirthDate"], inplace=True)

print(merged.shape)
print(df.shape)

(6783876, 94)
(6781880, 94)


In [17]:
# Among 
print(df["Voters_BirthDate"][0])
print(df["Voters_CalculatedRegDate"][0])

1997-05-06
2016-02-14


In [18]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def exact_voting_eligibility_date(birth_date_str):
    """
    Calculate the exact date a person becomes eligible to vote in a presidential election.
    
    Parameters:
    - birth_date_str: Date of birth in 'YYYY-MM-DD' format.
    
    Returns:
    - Date in 'YYYY-MM-DD' format when the person turns 18.
    """
    # Convert string to date object
    birth_date = datetime.strptime(birth_date_str, '%Y-%m-%d')
    
    # Calculate the exact date the person turns 18
    eligibility_date = birth_date + relativedelta(years=18)
    
    return eligibility_date.strftime("%Y-%m-%d")

# Apply the function to the DataFrame
df["vote_eligible_from"] = df["Voters_BirthDate"].map(exact_voting_eligibility_date)


In [19]:
# Test
index = 0
print(df["Voters_BirthDate"][index])
print(df["vote_eligible_from"][index])
print(df["General_2000_11_07"][index])
print(df["General_2004_11_02"][index])
print(df["General_2008_11_04"][index])
print(df["General_2012_11_06"][index])
print(df["General_2016_11_08"][index])
print(df["General_2020_11_03"][index])

1997-05-06
2015-05-06
nan
nan
nan
nan
Y
Y


In [20]:
# Test if any column is strange
election_col_dicts = {
    "2000-11-07": "General_2000_11_07",
    "2004-11-02": "General_2004_11_02",
    "2008-11-04": "General_2008_11_04",
    "2012-11-06": "General_2012_11_06",
    "2016-11-08": "General_2016_11_08",
    "2020-11-03": "General_2020_11_03"
}

col_keys = election_col_dicts.keys()
min_key = "2000-11-07"

strange_idxs = []

print(election_col_dicts.values())
df_small = df[list(election_col_dicts.values()) + ["vote_eligible_from", "LALVOTERID"]].copy()

strange_voter_ids = [] 

for idx, el in df_small.iterrows():
    eligible_from = el["vote_eligible_from"]
    voter_id = el["LALVOTERID"]
    if eligible_from <= min_key:
        continue
    cols_to_check = [election_col_dicts[key] for key in col_keys if key < eligible_from]
    if not el[cols_to_check].isna().all():
        print(f"Row, {idx} is strange. Check it")
        strange_idxs.append(idx)
        strange_voter_ids.append(voter_id)

dict_values(['General_2000_11_07', 'General_2004_11_02', 'General_2008_11_04', 'General_2012_11_06', 'General_2016_11_08', 'General_2020_11_03'])
Row, 50105 is strange. Check it
Row, 123576 is strange. Check it
Row, 133117 is strange. Check it
Row, 306844 is strange. Check it
Row, 410774 is strange. Check it
Row, 488549 is strange. Check it
Row, 535383 is strange. Check it
Row, 543398 is strange. Check it
Row, 596036 is strange. Check it
Row, 816871 is strange. Check it
Row, 903046 is strange. Check it
Row, 941576 is strange. Check it
Row, 1199939 is strange. Check it
Row, 1234408 is strange. Check it
Row, 1448259 is strange. Check it
Row, 2283303 is strange. Check it
Row, 2286156 is strange. Check it
Row, 2495327 is strange. Check it
Row, 2536611 is strange. Check it
Row, 2541233 is strange. Check it
Row, 2688279 is strange. Check it
Row, 2974737 is strange. Check it
Row, 3137265 is strange. Check it
Row, 3199054 is strange. Check it
Row, 3213243 is strange. Check it
Row, 3291757 is s

In [21]:
print(len(strange_idxs))
print(strange_idxs)

42
[50105, 123576, 133117, 306844, 410774, 488549, 535383, 543398, 596036, 816871, 903046, 941576, 1199939, 1234408, 1448259, 2283303, 2286156, 2495327, 2536611, 2541233, 2688279, 2974737, 3137265, 3199054, 3213243, 3291757, 3296871, 3314602, 4337609, 4845516, 5083666, 5134214, 5155097, 5192982, 5458030, 5598967, 5624236, 5735211, 6009923, 6033418, 6164286, 6532899]


In [11]:
filtered = df_small[df_small["LALVOTERID"].isin(strange_voter_ids)]
filtered

,General_2000_11_07,General_2004_11_02,General_2008_11_04,General_2012_11_06,General_2016_11_08,General_2020_11_03,vote_eligible_from,LALVOTERID
50105,NaN,Y,NaN,Y,Y,Y,2004-11-13,LALOH441796241
123576,Y,Y,Y,Y,Y,Y,2000-11-09,LALOH582487020
133117,NaN,Y,Y,Y,NaN,Y,2004-11-25,LALOH6000272
306844,NaN,NaN,NaN,NaN,Y,NaN,2016-11-25,LALOH495972207
410774,NaN,NaN,NaN,NaN,Y,Y,2016-12-20,LALOH484014325
488549,NaN,NaN,NaN,NaN,Y,Y,2016-11-20,LALOH484030587
535383,NaN,NaN,NaN,NaN,Y,Y,2016-11-27,LALOH484124347
543398,NaN,NaN,NaN,Y,Y,Y,2012-11-13,LALOH164567116
596036,NaN,NaN,Y,Y,Y,Y,2008-11-16,LALOH472635174
816871,NaN,NaN,NaN,NaN,Y,Y,2016-11-27,LALOH484116637


In [12]:
print(df.shape)
# Identify indices to drop
indices_to_drop = df[df["LALVOTERID"].isin(strange_voter_ids)].index

# Drop the rows by index
df_filtered = df.drop(index=indices_to_drop)
print(df_filtered.shape)


(6781880, 95)
(6781838, 95)


In [ ]:
# delete 
# df_filtered["can_vote_2000-11-07"] = df_filtered["vote_eligible_from"] >= "2000-11-07"
# df_filtered["can_vote_2004-11-02"] = df_filtered["vote_eligible_from"] >= "2004-11-02"

can_vote_columns = df_filtered.columns[df_filtered.columns.str.contains("can_vote")]
print(can_vote_columns)

for col in can_vote_columns:
    date = col.split("_")[-1]
    df_filtered[col] = df_filtered["vote_eligible_from"] <= date

Index(['can_vote_2000-11-07', 'can_vote_2004-11-02', 'can_vote_2008-11-04',
       'can_vote_2012-11-06', 'can_vote_2016-11-08', 'can_vote_2020-11-03'],
      dtype='object')


In [14]:
most_recent_vote_date = can_vote_columns[-1].split("_")[-1]

reference_date = datetime(2020, 11, 3)
# use date time difference days 
df["vote_life"] = (reference_date - pd.to_datetime(df["vote_eligible_from"])).dt.days

In [15]:
df_filtered.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'LALVOTERID',
       'Residence_Addresses_CensusBlock', 'Residence_Families_FamilyID',
       'Residence_Addresses_Property_Land_Square_Footage',
       'Residence_Addresses_Property_Type', 'Mailing_Addresses_State',
       'Mailing_Addresses_Zip', 'Voters_Gender', 'Voters_Age',
       'Voters_BirthDate', 'ConsumerData_Education_of_Person',
       'EthnicGroups_EthnicGroup1Desc', 'ConsumerData_Inferred_HH_Rank',
       'ConsumerData_Language_Code', 'ConsumerData_Marital_Status',
       'ConsumerData_Single_Parent', 'ConsumerDataLL_Veteran',
       'ConsumerData_African_American_Professional',
       'ConsumerData_Business_Owner', 'ConsumerData_Occupation_Group',
       'ConsumerData_Occupation_of_Person',
       'ConsumerData_LikelyRecentEmployment_Executive_Level',
       'ConsumerData_LikelyRecentEmployment_Executive_Department',
       'Voters_PlaceOfBirth', 'Voters_Active', 'Voters_CalculatedRegDate',
       'Voters_OfficialRegDate', 'Parties_De

In [16]:
# save df_filtered

df_filtered.to_csv("../data/oh_filtered_till_2020.csv")